In [2]:
%pip install numpy --upgrade
%pip install scikit-learn --upgrade
%pip install dask
%pip install imblearn


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# python library imports

import pandas as pd
import glob
import numpy as np
import dask.dataframe as dd
import gc
import sys
import numpy as np
import pyarrow
from sklearn.model_selection import train_test_split
import sagemaker
import os
import boto3
from imblearn.over_sampling import SMOTENC

In [4]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker.Session().default_bucket()

prefix = 'ipinyou'
os.environ["AWS_REGION"] = region

print(f'Region : {region}')
print(f'IAM Role : {role}')
print(f'S3 Bucket : {bucket_name}')

# Get the current installed version of Sagemaker SDK, TensorFlow, Python, Boto3 and SMDebug
print(f'SageMaker Python SDK version : {sagemaker.__version__}')
print(f'Python version : {sys.version}')

Region : us-east-1
IAM Role : arn:aws:iam::725069941408:role/service-role/AmazonSageMaker-ExecutionRole-20230327T095833
S3 Bucket : sagemaker-us-east-1-725069941408
SageMaker Python SDK version : 2.145.0
Python version : 3.7.10 (default, Jun  4 2021, 14:48:32) 
[GCC 7.5.0]


In [18]:
!aws s3 ls

2023-03-09 18:29:26 cloudtrail-awslogs-725069941408-yctvp0fc-isengard-do-not-delete
2023-03-10 04:44:45 do-not-delete-gatedgarden-audit-725069941408
2023-03-24 18:31:34 ipenyou
2023-04-17 12:17:28 ipenyou-db-load
2023-03-09 18:33:06 recs-725069941408
2023-04-14 16:38:55 sagemaker-studio-725069941408-246o1hyqioxh
2023-03-27 15:01:25 sagemaker-studio-725069941408-h75yrhgjwlh
2023-03-27 15:00:09 sagemaker-studio-izuds4ml68
2023-03-27 14:58:59 sagemaker-studio-o10z8bk5u59
2023-03-27 15:06:12 sagemaker-us-east-1-725069941408


In [21]:
!aws s3 cp data/training2nd s3://ipenyou-db-load --recursive

upload: data/training2nd/bid.20130610.txt to s3://ipenyou-db-load/bid.20130610.txt     
upload: data/training2nd/bid.20130608.txt to s3://ipenyou-db-load/bid.20130608.txt   
upload: data/training2nd/bid.20130606.txt to s3://ipenyou-db-load/bid.20130606.txt   
upload: data/training2nd/clk.20130606.txt to s3://ipenyou-db-load/clk.20130606.txt   
upload: data/training2nd/clk.20130607.txt to s3://ipenyou-db-load/clk.20130607.txt   
upload: data/training2nd/clk.20130608.txt to s3://ipenyou-db-load/clk.20130608.txt   
upload: data/training2nd/clk.20130609.txt to s3://ipenyou-db-load/clk.20130609.txt   
upload: data/training2nd/clk.20130610.txt to s3://ipenyou-db-load/clk.20130610.txt   
upload: data/training2nd/clk.20130611.txt to s3://ipenyou-db-load/clk.20130611.txt   
upload: data/training2nd/clk.20130612.txt to s3://ipenyou-db-load/clk.20130612.txt   
upload: data/training2nd/conv.20130606.txt to s3://ipenyou-db-load/conv.20130606.txt 
upload: data/training2nd/conv.20130607.txt to s3://i

In [29]:
!aws s3 cp data/user.profile.tags.en.txt s3://ipenyou-db-load

upload: data/user.profile.tags.en.txt to s3://ipenyou-db-load/user.profile.tags.en.txt


In [5]:
# define columns
ad_columns = ['BidID','Timestamp','Log Type','iPinYou ID','User-Agent','IP','Region','City','Ad Exchange','Domain','URL','Anonymous URL ID','Ad slot ID','Ad slot width','Ad slot height','Ad slot visibility','Ad slot format','Ad slot floor price','Creative ID','Bidding price','Paying price','Key page URL','Advertiser ID','User Tags']
ad_columns

['BidID',
 'Timestamp',
 'Log Type',
 'iPinYou ID',
 'User-Agent',
 'IP',
 'Region',
 'City',
 'Ad Exchange',
 'Domain',
 'URL',
 'Anonymous URL ID',
 'Ad slot ID',
 'Ad slot width',
 'Ad slot height',
 'Ad slot visibility',
 'Ad slot format',
 'Ad slot floor price',
 'Creative ID',
 'Bidding price',
 'Paying price',
 'Key page URL',
 'Advertiser ID',
 'User Tags']

In [6]:
ddf_imp = dd.read_csv('./data/training2nd/i*.txt',sep='\t',header=0,names=ad_columns)

In [7]:
ddf_clicks = dd.read_csv('./data/training2nd/clk*.txt',sep='\t',header=0,names=ad_columns)

In [8]:
clickids = ddf_clicks['BidID'].unique().compute()

In [9]:
print(f'A total of {len(ddf_imp)} impressions in the impression training dataset')

A total of 12237142 impressions in the impression training dataset


In [10]:
print(f'A total of {len(ddf_clicks)} clicks in the clicks training dataset')

A total of 9971 clicks in the clicks training dataset


In [11]:
df = ddf_imp

In [12]:
# create new boolean column for click 
df['clicks']=df['BidID'].isin(clickids)

In [13]:
print(f'A total of {len(df)} impressions in the impression training dataset')

A total of 12237142 impressions in the impression training dataset


In [14]:
df.loc[0].compute()

,BidID,Timestamp,Log Type,iPinYou ID,User-Agent,IP,Region,City,Ad Exchange,Domain,...,Ad slot visibility,Ad slot format,Ad slot floor price,Creative ID,Bidding price,Paying price,Key page URL,Advertiser ID,User Tags,clicks
0,5bd0cbeb2f82fb94e56b7dc2e6b77ec,20130606000104252,1,Vhkr1uaROqKsXmb,Mozilla/4.0 (compatible; MSIE 7.0; Windows NT ...,222.220.35.*,308,320,2,eSMvBpa0jqmUagk4JKTI,...,2,0,5,13606a7c541dcd9ca1948875a760bb31,238,72,d29e59bf0f7f8243858b8183f14d4412,3358,"13800,10024",False
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
0,8a15b98c8f9e60d4f92aaab01acf52a4,20130606000104192,1,VhTVORqG36N6qMj,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...,114.100.37.*,106,117,1,lsxSl559Xql7FmMs,...,0,1,0,23d6dade7ed21cea308205b37594003e,227,207,b2e35064f3549d447edbbdfb1f707c8c,3427,"10063,10684,10083,13403,10059,10024,10048,1005...",False
